# **Big Data Intelligence Project - TMALL Repeat Buyers**
### **Armando Fortes, David Pissarra, Gabriele Oliaro**

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, DMatrix, train
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

### Constants and Hyperparameters

In [2]:
DATA_DIR_1 = '../data_format1/'
DATA_DIR_2 = '../data_format2/'
TRAIN_PATH = DATA_DIR_1 + 'train_format1.csv'
TEST_PATH = DATA_DIR_1 + 'test_format1.csv'
USER_INFO_PATH = DATA_DIR_1 + 'user_info_format1.csv'
USER_LOG_PATH = DATA_DIR_1 + 'user_log_format1.csv'
DOUBLE11_DAY = 184

In [75]:
VALID_SET_SIZE = 0.2
PCA_COMPONENTS = 5
RANDOM_SEED = 42
EPSILON = 1e-10
SPLITS = 10

## **Data Pre-Processing**

### Read CSVs

In [4]:
df_train = pd.read_csv(TRAIN_PATH)
df_user_info = pd.read_csv(USER_INFO_PATH)
df_user_log = pd.read_csv(USER_LOG_PATH)
df_test = pd.read_csv(TEST_PATH)
df_test.drop('prob', axis=1, inplace=True)

df_train['kind'] = 'train'
df_test['kind'] = 'test'
df = df_train.append(df_test)

### Optimize memory usage and Data Cleaning

In [5]:
print(f'{round(df_user_log.memory_usage().sum() / 2**30, 2)} GB')

2.86 GB


In [6]:
df_user_log['user_id'] = df_user_log['user_id'].astype(np.int32)
df_user_log['item_id'] = df_user_log['item_id'].astype(np.int32)
df_user_log['cat_id'] = df_user_log['cat_id'].astype(np.int16)
df_user_log['seller_id'] = df_user_log['seller_id'].astype(np.int16)
df_user_log.rename(columns={'seller_id' : 'merchant_id'}, inplace=True)
df_user_log['brand_id'].fillna(0, inplace=True)
df_user_log['brand_id'] = df_user_log['brand_id'].astype(np.int16)
df_user_log['time_stamp'] = (pd.to_datetime(df_user_log['time_stamp'], format='%m%d') - pd.to_datetime(df_user_log['time_stamp'].min(), format='%m%d')).dt.days
df_user_log['time_stamp'] = df_user_log['time_stamp'].astype(np.int16)
df_user_log['action_type'] = df_user_log['action_type'].astype(np.int8)

In [7]:
print(f'{round(df_user_log.memory_usage().sum() / 2**30, 2)} GB')

0.87 GB


In [8]:
df_user_info['age_range'].fillna(0, inplace=True)
df_user_info['gender'].fillna(2, inplace=True)
df_user_info['age_range'] = df_user_info['age_range'].astype(np.int8)
df_user_info['gender'] = df_user_info['gender'].astype(np.int8)

## **Feature Engineering**

In [9]:
users = df_user_log.groupby('user_id')
merchants = df_user_log.groupby('merchant_id')
users_merchants = df_user_log.groupby(['user_id', 'merchant_id'])

double11 = (df_user_log[df_user_log['time_stamp'] == DOUBLE11_DAY]).reset_index(drop=True)
double11_users = double11.groupby('user_id')
double11_merchants = double11.groupby('merchant_id')
double11_users_merchants = double11.groupby(['user_id', 'merchant_id'])

### General counting and ratio features

In [10]:
# transform age categorical features into different binary features
to_merge = pd.get_dummies(df_user_info, prefix='age', columns=['age_range'])
df = df.merge(to_merge, on='user_id', how='left')

# count total number of unique values from each feature for a given user 
to_merge = users.nunique().reset_index().rename(columns={
    'item_id': 'items_user', 
    'cat_id': 'categories_user',
    'merchant_id': 'merchants_user',
    'brand_id': 'brands_user',
    'time_stamp': 'dates_user',
    'action_type': 'action_types_user'
    })
df = df.merge(to_merge, on='user_id', how='left')

# count total number of unique values from each feature for a given merchant 
to_merge = merchants.nunique().reset_index().rename(columns={
    'item_id': 'items_merchant', 
    'cat_id': 'categories_merchant',
    'user_id': 'userss_merchant',
    'brand_id': 'brands_merchant',
    'time_stamp': 'dates_merchant',
    'action_type': 'action_types_merchant'
    })
df = df.merge(to_merge, on='merchant_id', how='left')

# count total number of unique values from each feature for a given user and merchant
to_merge = users_merchants.nunique().reset_index().rename(columns={
    'item_id': 'items_user_merchant', 
    'cat_id': 'categories_user_merchant',
    'brand_id': 'brands_user_merchant',
    'time_stamp': 'dates_user_merchant',
    'action_type': 'action_types_user_merchant'
    })
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

# count total actions by type for a given user
to_merge = users['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_user',
    1: 'carts_user',
    2: 'purchases_user',
    3: 'favourites_user'
    })
df = df.merge(to_merge, on='user_id', how='left')

# count total actions by type for a given merchant
to_merge = merchants['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_merchant', 
    1: 'carts_merchant',
    2: 'purchases_merchant',
    3: 'favourites_merchant'
    })
df = df.merge(to_merge, on='merchant_id', how='left')

# count total actions by type for a given pair (user, merchant)
to_merge = users_merchants['action_type'].value_counts().unstack(fill_value=0).rename(columns={
    0: 'clicks_user_merchant',
    1: 'carts_user_merchant',
    2: 'purchases_user_merchant',
    3: 'favourites_user_merchant'
    })
df = df.merge(to_merge, on=['user_id', 'merchant_id'], how='left')

# ratio of actions in each merchant (user perspective)
df['clicks_in_merchant_ratio_perspective'] = df['clicks_user_merchant'] / (df['clicks_user'] + EPSILON)
df['carts_in_merchant_ratio_perspective'] = df['carts_user_merchant'] / (df['carts_user'] + EPSILON)
df['purchases_in_merchant_ratio_perspective'] = df['purchases_user_merchant'] / (df['purchases_user'] + EPSILON)
df['favourites_in_merchant_ratio_perspective'] = df['favourites_user_merchant'] / (df['favourites_user'] + EPSILON)

# ratio of actions in each merchant (merchant perspective)
df['clicks_by_user_ratio_perspective'] = df['clicks_user_merchant'] / (df['clicks_merchant'] + EPSILON)
df['carts_by_user_ratio_perspective'] = df['carts_user_merchant'] / (df['carts_merchant'] + EPSILON)
df['purchases_by_user_ratio_perspective'] = df['purchases_user_merchant'] / (df['purchases_merchant'] + EPSILON)
df['favourites_by_user_ratio_perspective'] = df['favourites_user_merchant'] / (df['favourites_merchant'] + EPSILON)

# ratio of each action type for a given user
df['clicks_user_ratio'] = df['clicks_user'] / (df['clicks_user'] + df['carts_user'] + df['purchases_user'] + df['favourites_user'] + EPSILON)
df['carts_user_ratio'] = df['carts_user'] / (df['clicks_user'] + df['carts_user'] + df['purchases_user'] + df['favourites_user'] + EPSILON)
df['purchases_user_ratio'] = df['purchases_user'] / (df['clicks_user'] + df['carts_user'] + df['purchases_user'] + df['favourites_user'] + EPSILON)
df['favourites_user_ratio'] = df['favourites_user'] / (df['clicks_user'] + df['carts_user'] + df['purchases_user'] + df['favourites_user'] + EPSILON)

# ratio of each action type for a given merchant
df['clicks_merchant_ratio'] = df['clicks_merchant'] / (df['clicks_merchant'] + df['carts_merchant'] + df['purchases_merchant'] + df['favourites_merchant'] + EPSILON)
df['carts_merchant_ratio'] = df['carts_merchant'] / (df['clicks_merchant'] + df['carts_merchant'] + df['purchases_merchant'] + df['favourites_merchant'] + EPSILON)
df['purchases_merchant_ratio'] = df['purchases_merchant'] / (df['clicks_merchant'] + df['carts_merchant'] + df['purchases_merchant'] + df['favourites_merchant'] + EPSILON)
df['favourites_merchant_ratio'] = df['favourites_merchant'] / (df['clicks_merchant'] + df['carts_merchant'] + df['purchases_merchant'] + df['favourites_merchant'] + EPSILON)

# ratio of each action type for a given pair (user, merchant)
df['clicks_user_merchant_ratio'] = df['clicks_user_merchant'] / (df['clicks_user_merchant'] + df['carts_user_merchant'] + df['purchases_user_merchant'] + df['favourites_user_merchant'] + EPSILON)
df['carts_user_merchant_ratio'] = df['carts_user_merchant'] / (df['clicks_user_merchant'] + df['carts_user_merchant'] + df['purchases_user_merchant'] + df['favourites_user_merchant'] + EPSILON)
df['purchases_user_merchant_ratio'] = df['purchases_user_merchant'] / (df['clicks_user_merchant'] + df['carts_user_merchant'] + df['purchases_user_merchant'] + df['favourites_user_merchant'] + EPSILON)
df['favourites_user_merchant_ratio'] = df['favourites_user_merchant'] / (df['clicks_user_merchant'] + df['carts_user_merchant'] + df['purchases_user_merchant'] + df['favourites_user_merchant'] + EPSILON)

# interval features
to_merge = (users['time_stamp'].max() - users['time_stamp'].min()).rename('interval')
df = df.merge(to_merge, on='user_id', how='left')

### Double11 Features

In [11]:
double11_day = 184
to_merge = (df_user_log[df_user_log['time_stamp'] == double11_day]).reset_index(drop=True)
to_merge = to_merge[to_merge['action_type'] == 2].reset_index(drop=True).groupby('user_id').size().reset_index()
df = df.merge(to_merge, on='user_id', how='left').rename(columns={0: 'double11_purchases'})

df['double11_ratio'] = df['double11_purchases'] / df['purchases_user']

### Max, Mean, Standard deviation and Median on user-merchant actions (grouping by users)

In [12]:
clmns = ['clicks_user_merchant', 'carts_user_merchant', 'purchases_user_merchant', 'favourites_user_merchant']

to_merge = df.groupby('user_id')[clmns].max().rename(columns={
    'clicks_user_merchant': 'clicks_user_merchant_max',
    'carts_user_merchant': 'carts_user_merchant_max',
    'purchases_user_merchant': 'purchases_user_merchant_max',
    'favourites_user_merchant': 'favourites_user_merchant_max'
})
df = df.merge(to_merge, on='user_id', how='left')

to_merge = df.groupby('user_id')[clmns].mean().rename(columns={
    'clicks_user_merchant': 'clicks_user_merchant_mean',
    'carts_user_merchant': 'carts_user_merchant_mean',
    'purchases_user_merchant': 'purchases_user_merchant_mean',
    'favourites_user_merchant': 'favourites_user_merchant_mean'
})
df = df.merge(to_merge, on='user_id', how='left')

to_merge = df.groupby('user_id')[clmns].std().rename(columns={
    'clicks_user_merchant': 'clicks_user_merchant_std',
    'carts_user_merchant': 'carts_user_merchant_std',
    'purchases_user_merchant': 'purchases_user_merchant_std',
    'favourites_user_merchant': 'favourites_user_merchant_std'
}).fillna(0)
df = df.merge(to_merge, on='user_id', how='left')

to_merge = df.groupby('user_id')[clmns].median().rename(columns={
    'clicks_user_merchant': 'clicks_user_merchant_median',
    'carts_user_merchant': 'carts_user_merchant_median',
    'purchases_user_merchant': 'purchases_user_merchant_median',
    'favourites_user_merchant': 'favourites_user_merchant_median'
})
df = df.merge(to_merge, on='user_id', how='left')

### Max, Mean, Standard deviation and Median on user-merchant actions (grouping by merchants)

In [13]:
to_merge = df.groupby('merchant_id')[clmns].max().rename(columns={
    'clicks_user_merchant': 'clicks_merchant_user_max',
    'carts_user_merchant': 'carts_merchant_user_max',
    'purchases_user_merchant': 'purchases_merchant_user_max',
    'favourites_user_merchant': 'favourites_merchant_user_max'
})
df = df.merge(to_merge, on='merchant_id', how='left')

to_merge = df.groupby('merchant_id')[clmns].mean().rename(columns={
    'clicks_user_merchant': 'clicks_merchant_user_mean',
    'carts_user_merchant': 'carts_merchant_user_mean',
    'purchases_user_merchant': 'purchases_merchant_user_mean',
    'favourites_user_merchant': 'favourites_merchant_user_mean'
})
df = df.merge(to_merge, on='merchant_id', how='left')

to_merge = df.groupby('merchant_id')[clmns].std().rename(columns={
    'clicks_user_merchant': 'clicks_merchant_user_std',
    'carts_user_merchant': 'carts_merchant_user_std',
    'purchases_user_merchant': 'purchases_merchant_user_std',
    'favourites_user_merchant': 'favourites_merchant_user_std'
}).fillna(0)
df = df.merge(to_merge, on='merchant_id', how='left')

to_merge = df.groupby('merchant_id')[clmns].median().rename(columns={
    'clicks_user_merchant': 'clicks_merchant_user_median',
    'carts_user_merchant': 'carts_merchant_user_median',
    'purchases_user_merchant': 'purchases_merchant_user_median',
    'favourites_user_merchant': 'favourites_merchant_user_median'
})
df = df.merge(to_merge, on='merchant_id', how='left')

### PCA Features

In [14]:
# PCA features
pca_df = df.drop(['kind', 'label'], axis=1)
pca = PCA(n_components=PCA_COMPONENTS)
pca.fit(pca_df)
df = df.join(pd.DataFrame(pca.transform(pca_df), index=pca_df.index).add_prefix('pca_'))

### Splitting into Training and Validation Sets

In [60]:
df_train = df[df['kind'] == 'train'].drop(['kind'], axis=1)
df_test = df[df['kind'] == 'test'].drop(['kind', 'label'], axis=1)

X, y = df_train.drop(columns='label'), df_train['label']

In [61]:
X

,user_id,merchant_id,gender,age_0,age_1,age_2,age_3,age_4,age_5,age_6,...,favourites_merchant_user_std,clicks_merchant_user_median,carts_merchant_user_median,purchases_merchant_user_median,favourites_merchant_user_median,pca_0,pca_1,pca_2,pca_3,pca_4
0,34176,3906,0,0,0,0,0,0,0,1,...,2.630425,8.0,0.0,1.0,0.0,-178099.961509,-36617.862766,211.660468,-85.973362,-2110.210403
1,34176,121,0,0,0,0,0,0,0,1,...,1.430079,8.0,0.0,1.0,0.0,-177681.074681,21074.149225,-6.040211,700.193444,2994.062130
2,34176,4356,0,0,0,0,0,0,0,1,...,0.333333,2.5,0.0,1.0,0.0,-178166.109770,-45742.449912,-1997.565927,-1401.338012,-1228.393809
3,34176,2217,0,0,0,0,0,0,0,1,...,1.166602,4.0,0.0,1.0,0.0,-177820.329980,1921.002882,7488.545356,1196.754418,-1017.657227
4,230784,4818,0,1,0,0,0,0,0,0,...,4.435505,10.0,0.0,1.0,0.0,18709.568378,-9577.743348,-673.574943,-2083.139971,-1065.529847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260859,359807,4325,1,0,0,0,0,1,0,0,...,0.560529,4.0,0.0,1.0,0.0,147535.705684,-37164.587706,96.604558,-1535.748663,-1011.011148
260860,294527,3971,1,1,0,0,0,0,0,0,...,0.719238,3.0,0.0,1.0,0.0,82316.355289,-28568.709834,1450.838118,-1160.843016,-905.215611
260861,294527,152,1,1,0,0,0,0,0,0,...,0.638111,3.0,0.0,1.0,0.0,82183.492873,-46879.577205,-1991.979462,2026.349289,1224.299090
260862,294527,2537,1,1,0,0,0,0,0,0,...,0.953802,4.0,0.0,1.0,0.0,82427.860167,-13213.242991,3914.757352,-1603.598163,1894.539409


## **Models**

### Fetch best features
The xgboost object will train the model with all features, then the booster object (returned after training) can calculate which features best contribute for most information gain.

In [62]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=VALID_SET_SIZE, random_state=RANDOM_SEED)

dtrain = DMatrix(X_train, label=y_train)
dvalid = DMatrix(X_valid, label=y_valid)
watchlist = [(dvalid, 'valid')]
params = {
    'max_depth': 7,
    'min_child_weight': 200, 
    'colsample_bytree': 0.8, 
    'subsample': 0.8, 
    'eta': 0.04,    
    'seed': RANDOM_SEED,
    'eval_metric': 'auc'
}
booster = train(params, dtrain, num_boost_round=2000, evals=watchlist, early_stopping_rounds=50)
best_features = pd.DataFrame(booster.get_score(importance_type='gain').items(), columns=['features', 'importance'])['features'].to_numpy()
X, X_train, X_valid = X[best_features], X_train[best_features], X_valid[best_features]

[0]	valid-auc:0.64009
[1]	valid-auc:0.64988
[2]	valid-auc:0.65324
[3]	valid-auc:0.65436
[4]	valid-auc:0.65772
[5]	valid-auc:0.65846
[6]	valid-auc:0.65927
[7]	valid-auc:0.66009
[8]	valid-auc:0.66053
[9]	valid-auc:0.66188
[10]	valid-auc:0.66236
[11]	valid-auc:0.66355
[12]	valid-auc:0.66379
[13]	valid-auc:0.66510
[14]	valid-auc:0.66666
[15]	valid-auc:0.66723
[16]	valid-auc:0.66791
[17]	valid-auc:0.66860
[18]	valid-auc:0.66937
[19]	valid-auc:0.66958
[20]	valid-auc:0.67060
[21]	valid-auc:0.67117
[22]	valid-auc:0.67188
[23]	valid-auc:0.67208
[24]	valid-auc:0.67253
[25]	valid-auc:0.67261
[26]	valid-auc:0.67308
[27]	valid-auc:0.67314
[28]	valid-auc:0.67326
[29]	valid-auc:0.67319
[30]	valid-auc:0.67380
[31]	valid-auc:0.67421
[32]	valid-auc:0.67409
[33]	valid-auc:0.67430
[34]	valid-auc:0.67459
[35]	valid-auc:0.67476
[36]	valid-auc:0.67484
[37]	valid-auc:0.67590
[38]	valid-auc:0.67660
[39]	valid-auc:0.67708
[40]	valid-auc:0.67727
[41]	valid-auc:0.67758
[42]	valid-auc:0.67771
[43]	valid-auc:0.6779

In [63]:
X

,user_id,merchant_id,gender,age_0,age_2,age_3,age_4,age_5,age_6,items_user,...,carts_merchant_user_std,purchases_merchant_user_std,favourites_merchant_user_std,clicks_merchant_user_median,purchases_merchant_user_median,pca_0,pca_1,pca_2,pca_3,pca_4
0,34176,3906,0,0,0,0,0,0,1,256,...,0.263884,0.640641,2.630425,8.0,1.0,-178099.961509,-36617.862766,211.660468,-85.973362,-2110.210403
1,34176,121,0,0,0,0,0,0,1,256,...,0.283098,1.269946,1.430079,8.0,1.0,-177681.074681,21074.149225,-6.040211,700.193444,2994.062130
2,34176,4356,0,0,0,0,0,0,1,256,...,0.125000,1.173344,0.333333,2.5,1.0,-178166.109770,-45742.449912,-1997.565927,-1401.338012,-1228.393809
3,34176,2217,0,0,0,0,0,0,1,256,...,0.154593,0.400811,1.166602,4.0,1.0,-177820.329980,1921.002882,7488.545356,1196.754418,-1017.657227
4,230784,4818,0,1,0,0,0,0,0,31,...,0.418602,1.192333,4.435505,10.0,1.0,18709.568378,-9577.743348,-673.574943,-2083.139971,-1065.529847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260859,359807,4325,1,0,0,0,1,0,0,49,...,0.214524,0.667922,0.560529,4.0,1.0,147535.705684,-37164.587706,96.604558,-1535.748663,-1011.011148
260860,294527,3971,1,1,0,0,0,0,0,89,...,0.235101,0.544233,0.719238,3.0,1.0,82316.355289,-28568.709834,1450.838118,-1160.843016,-905.215611
260861,294527,152,1,1,0,0,0,0,0,89,...,0.112905,0.432524,0.638111,3.0,1.0,82183.492873,-46879.577205,-1991.979462,2026.349289,1224.299090
260862,294527,2537,1,1,0,0,0,0,0,89,...,0.282897,0.960676,0.953802,4.0,1.0,82427.860167,-13213.242991,3914.757352,-1603.598163,1894.539409


In [64]:
X, X_train, X_valid = X.to_numpy(), X_train.to_numpy(), X_valid.to_numpy()
y, y_train, y_valid = y.to_numpy(), y_train.to_numpy(), y_valid.to_numpy()

### Models Setup

In [73]:
# TODO tuning
models = {
    # 'RandomForestClassifier': RandomForestClassifier(
    #     oob_score=True, 
    #     n_estimators=1000, 
    #     max_depth=10, 
    #     max_features='sqrt',
    #     n_jobs=-1
    # ),
    # 'CatBoostClassifier': CatBoostClassifier(
    #     depth=6,
    #     learning_rate=0.05,
    #     iterations=1200,
    #     eval_metric='AUC',
    #     random_state=RANDOM_SEED,
    #     thread_count=8,
    #     silent=True
    # ),
    # 'LGBMClassifier': LGBMClassifier(
    #     n_estimators=2000,
    #     max_depth=8,
    #     num_leaves=50,
    #     learning_rate=0.03,
    #     reg_lambda=1,
    #     objective='binary',
    #     metric=['auc'],
    #     random_state=RANDOM_SEED,
    #     n_jobs=-1
    # ),
    'XGBClassifier': [XGBClassifier, {
        'max_depth': 7,
        'n_estimators': 1000,
        'min_child_weight': 200,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
        'eta': 0.04,
        'objective': 'binary:logistic',
        'use_label_encoder': False,
        'seed': RANDOM_SEED
    }]
}

### Training + Cross Validation (10Fold)

In [31]:
# fit_params = {
#     'RandomForestClassifier': {}, 
#     'CatBoostClassifier': {},
#     'LGBMClassifier': {},
#     'XGBClassifier': {
#         'eval_metric': 'auc',
#         'eval_set': [(X_train, y_train), (X_valid, y_valid)],
#         'early_stopping_rounds': 20,
#         'verbose': False
#     }
# }

# for name, params in model_params.items():

#     model = name(**params)
#     model.fit(X_train, y_train, **fit_params[name])
#     predictions = model.predict_proba(X_valid)[:,1]
#     roc_auc = roc_auc_score(y_valid, predictions)

#     print('% ' + name + ': ' + '{0:.4f}'.format(roc_auc))

=================================== ROC AUC Scores ===================================
% CatBoostClassifier: 0.6889
% LGBMClassifier: 0.6750
% XGBClassifier: 0.6922


In [76]:
records = {}

for name, model_params in models.items():

    _class, params = model_params
    model_records = {'best_score': 0, 'scores': []}

    kf = KFold(n_splits=SPLITS, shuffle=True, random_state=RANDOM_SEED)

    for train_index, valid_index in kf.split(X):
    
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]

        model = _class(**params)

        fit_params = {
            'RandomForestClassifier': {}, 
            'CatBoostClassifier': {},
            'LGBMClassifier': {},
            'XGBClassifier': {
                'eval_metric': 'auc',
                'eval_set': [(X_train, y_train), (X_valid, y_valid)],
                'early_stopping_rounds': 50,
                'verbose': True
            }
        }

        model.fit(X_train, y_train, **fit_params[name])
        predictions = model.predict_proba(X_valid)[:,1]
        model_records['scores'].append(roc_auc_score(y_valid, predictions))
        if model_records['scores'][-1] > model_records['best_score']:
            model_records['best_score'] = model_records['scores'][-1]
            model_records['best_instance'] = model

    records[name] = model_records

    print(f'% {name} %')
    print('mean score: {0:.4f}'.format(np.mean(model_records['scores'])))
    print('best score: {0:.4f}'.format(model_records['best_score']))

[0]	validation_0-auc:0.64802	validation_1-auc:0.63824
[1]	validation_0-auc:0.65603	validation_1-auc:0.64693
[2]	validation_0-auc:0.66150	validation_1-auc:0.64956
[3]	validation_0-auc:0.66535	validation_1-auc:0.65267
[4]	validation_0-auc:0.66623	validation_1-auc:0.65281
[5]	validation_0-auc:0.66663	validation_1-auc:0.65366
[6]	validation_0-auc:0.66700	validation_1-auc:0.65529
[7]	validation_0-auc:0.66693	validation_1-auc:0.65434
[8]	validation_0-auc:0.66770	validation_1-auc:0.65475
[9]	validation_0-auc:0.66975	validation_1-auc:0.65623
[10]	validation_0-auc:0.66993	validation_1-auc:0.65617
[11]	validation_0-auc:0.66975	validation_1-auc:0.65668
[12]	validation_0-auc:0.67086	validation_1-auc:0.65686
[13]	validation_0-auc:0.67178	validation_1-auc:0.65834
[14]	validation_0-auc:0.67178	validation_1-auc:0.65824
[15]	validation_0-auc:0.67225	validation_1-auc:0.65830
[16]	validation_0-auc:0.67302	validation_1-auc:0.65844
[17]	validation_0-auc:0.67339	validation_1-auc:0.65833
[18]	validation_0-au

## **Submission**

### Ensemble Predictions from Best Model Instances

In [78]:
prob_submission = records['XGBClassifier']['best_instance'].predict_proba(df_test[best_features].to_numpy())[:,1]
submission = df_test.iloc[:,:2].join(pd.DataFrame(prob_submission, index=df_test.index).rename(columns={0:'prob'}))
submission.to_csv('submission.csv', index=False)

0.7474897547782006
